In [48]:
from nemo.collections.asr.models.aed_multitask_models import EncDecMultiTaskModel, MultiTaskTranscriptionConfig
import librosa
from IPython.display import Audio

In [49]:
canary:EncDecMultiTaskModel = EncDecMultiTaskModel.from_pretrained(model_name="nvidia/canary-180m-flash", map_location="cpu").eval() # type: ignore

[NeMo I 2026-01-07 16:19:30 nemo_logging:393] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2026-01-07 16:19:30 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1152 tokens
[NeMo I 2026-01-07 16:19:30 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-07 16:19:30 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-07 16:19:30 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-07 16:19:30 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-07 16:19:30 nemo_logging:393] Aggregate vocab size: 5248


[NeMo W 2026-01-07 16:19:31 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 8
    pin_memory: true
    prompt_format: canary2
    max_tps: 25
    max_duration: 40.0
    text_field: answer
    lang_field: target_lang
    use_bucketing: true
    bucket_duration_bins:
    - - 3.56
      - 30
    - - 3.56
      - 77
    - - 4.608
      - 38
    - - 4.608
      - 88
    - - 5.48
      - 49
    - - 5.48
      - 106
    - - 6.05
      - 52
    - - 6.05
      - 109
    - - 6.85
      - 54
    - - 6.85
      - 124
    - - 7.914
      - 59
    - - 7.914
      - 137
    - - 8.52
      - 67
    - - 8.52
      - 158
    - - 9.51
      - 67
    - - 9.51
      - 153
    - - 10.29
      -

[NeMo I 2026-01-07 16:19:31 nemo_logging:393] PADDING: 0


ERROR:hydra.utils:Error getting class at nemo.collections.asr.modules.transformer.get_nemo_transformer: Located non-class of type 'function' while loading 'nemo.collections.asr.modules.transformer.get_nemo_transformer'


[NeMo I 2026-01-07 16:19:32 nemo_logging:393] Model EncDecMultiTaskModel was successfully restored from /home/tassilo-holtzwart/.cache/huggingface/hub/models--nvidia--canary-180m-flash/snapshots/b12ab418510d093e83890178fd0e8b0d0f7918a6/canary-180m-flash.nemo.


In [55]:
import os
audio_paths = [os.path.join(root, file)
               for root, _, files in os.walk("audios")
               for file in files if file.endswith(('.wav', '.mp3'))]

In [56]:
import torch

audio_batch = [torch.tensor(librosa.load(ap, sr= 16000)[0]) for ap in audio_paths]

In [57]:
audio_lengths = [len(audio) for audio in audio_batch]

In [58]:
audios = torch.nn.utils.rnn.pad_sequence(audio_batch, batch_first=True)
lengths = torch.tensor(audio_lengths, dtype=torch.int64)

In [59]:
[display(Audio(data=audio.numpy(), rate=16000)) for audio in audio_batch]

[None, None, None, None, None, None, None, None]

In [60]:
canary.transcribe(audio_paths, source_lang='de', target_lang='de')

[NeMo W 2026-01-07 16:20:32 nemo_logging:405] Chunking is disabled. Please pass a single audio file or set batch_size to 1
[NeMo W 2026-01-07 16:20:32 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: enable_chunking,trim_silence
[NeMo W 2026-01-07 16:20:32 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 2it [00:05,  2.86s/it]


[Hypothesis(score=0.0, y_sequence=tensor([2215, 2359, 2505, 2218, 2252, 2212, 2529, 2178, 2609, 2728, 2390, 2212,
         2184, 2267, 2255, 2204, 2474, 2178, 2243, 2200, 2218, 2403, 2205, 2351,
         2440, 2184, 2459, 2866, 3062, 2327, 2611, 2197, 2955, 2400, 3054, 2218,
         2389, 2247, 2285, 2242, 2352, 2529, 2177, 2335, 2270, 2348, 2194, 2474,
         2763, 2228, 2299, 2290, 3075, 2223, 3051, 2412, 2213]), text=' Sadik , der in ihren Herzen wie in seinem eigenen las , hatte immer seine Freude daran gehabt , dem Wachstum ihrer gegenseitigen Neigung zuzusehen .', dec_out=None, dec_state=None, timestamp=[], alignments=None, frame_confidence=None, token_confidence=None, word_confidence=None, length=0, y=None, lm_state=None, lm_scores=None, ngram_lm_state=None, tokens=None, last_token=None, token_duration=None, last_frame=None),
 Hypothesis(score=0.0, y_sequence=tensor([2284, 2267, 2275, 2222, 2296, 2308, 2291, 2629, 2178, 2232, 2256, 2267,
         2965, 2194, 2262, 2213, 2569,

In [61]:
trcfg = MultiTaskTranscriptionConfig(
            batch_size=4,
            return_hypotheses=False,
            num_workers=0,
            channel_selector=None,
            augmentor=None,
            verbose=True,
            prompt={
                "source_lang": "de",
                "target_lang": "de"
            },
            timestamps=False,
        )

In [62]:
import warnings
prompt = [{'role': 'user', 'slots': {'source_lang': 'de', 'target_lang': 'de'}}]
default_turns = canary.prompt.get_default_dialog_slots()
turns = prompt.copy()  # shallow copy #1: don't override the config
for turn in turns:
    role = turn["role"]
    # Check if we have defaults for this role.
    # There shouldn't be more than a single turn for a given role, but if there are,
    # we'll emit a warning.
    if default_turns_for_role := [t for t in default_turns if t["role"] == role]:
        if len(default_turns_for_role) > 1:
            warnings.warn(
                f"More than one default turn detected for {role=}. "
                f"We'll be using default slot values for the first turn of {role=} only."
            )
        default_slots = default_turns_for_role[0]["slots"]
        turn["slots"] = turn["slots"].copy()  # shallow copy #1: don't override the config
        # fill missing slots using defaults
        for slot, val in default_slots.items():
            if turn["slots"].get(slot) is None:
                turn["slots"][slot] = val

decoder_input_ids = (
    canary.prompt.encode_dialog(turns=turns)["context_ids"]
    .unsqueeze(0)
    .repeat(len(audio_batch), 1)
    .to("cpu")
)

In [84]:
with torch.inference_mode():
    _, encoded_len, encoded, encoded_mask = canary(input_signal=audios, input_signal_length=lengths)

In [86]:
torch.save(encoded, "encoded8.pt")

In [64]:
canary.decoding.cfg

{'strategy': 'beam', 'return_best_hypothesis': True, 'beam': {'beam_size': 1, 'len_pen': 0.0, 'max_generation_delta': 50}}

In [65]:
canary.change_decoding_strategy({"strategy":"greedy", "return_best_hypothesis":True})

[NeMo I 2026-01-07 16:20:46 nemo_logging:393] Changed decoding strategy to 
    strategy: greedy
    compute_hypothesis_token_set: false
    preserve_alignments: null
    confidence_cfg:
      preserve_frame_confidence: false
      preserve_token_confidence: false
      preserve_word_confidence: false
      exclude_blank: true
      aggregation: min
      tdt_include_duration: false
      method_cfg:
        name: entropy
        entropy_type: tsallis
        alpha: 0.33
        entropy_norm: exp
        temperature: DEPRECATED
    compute_langs: false
    greedy:
      temperature: null
      max_generation_delta: -1
      preserve_alignments: false
      preserve_token_confidence: false
      confidence_method_cfg:
        name: entropy
        entropy_type: tsallis
        alpha: 0.33
        entropy_norm: exp
        temperature: DEPRECATED
      n_samples: 1
    beam:
      beam_size: 1
      search_type: default
      len_pen: 1.0
      max_generation_delta: -1
      return_best_

In [66]:
hypothesis, = canary.decoding.decoding(encoder_hidden_states=encoded, encoder_input_mask = encoded_mask, decoder_input_ids=decoder_input_ids)

In [67]:
prediction = hypothesis[0].y_sequence

In [17]:
canary.tokenizer.ids_to_text(prediction)

' Sadik , der in ihren Herzen wie in seinem eigenen las , hatte immer seine Freude daran gehabt , dem Wachstum ihrer gegenseitigen Neigung zuzusehen .'

In [18]:
for token in prediction:
    print(f"{token.item()}: {canary.tokenizer.ids_to_tokens([token.item()])[0]}")

2215: ▁S
2359: ad
2505: ik
2218: ▁,
2252: ▁der
2212: ▁in
2529: ▁ihr
2178: en
2609: ▁Her
2728: zen
2390: ▁wie
2212: ▁in
2184: ▁s
2267: ein
2255: em
2204: ▁e
2474: igen
2178: en
2243: ▁l
2200: as
2218: ▁,
2403: ▁hat
2205: te
2351: ▁im
2440: mer
2184: ▁s
2459: eine
2866: ▁Fre
3062: u
2327: de
2611: ▁dar
2197: an
2955: ▁geh
2400: ab
3054: t
2218: ▁,
2389: ▁dem
2247: ▁W
2285: ach
2242: st
2352: um
2529: ▁ihr
2177: er
2335: ▁ge
2270: gen
2348: se
2194: it
2474: igen
2763: ▁Ne
2228: ig
2299: ung
2290: ▁zu
3075: z
2223: us
3051: e
2412: hen
2213: ▁.


In [19]:
canary.decoding.decode_hypothesis(hypothesis)

[Hypothesis(score=0.0, y_sequence=tensor([2215, 2359, 2505, 2218, 2252, 2212, 2529, 2178, 2609, 2728, 2390, 2212,
         2184, 2267, 2255, 2204, 2474, 2178, 2243, 2200, 2218, 2403, 2205, 2351,
         2440, 2184, 2459, 2866, 3062, 2327, 2611, 2197, 2955, 2400, 3054, 2218,
         2389, 2247, 2285, 2242, 2352, 2529, 2177, 2335, 2270, 2348, 2194, 2474,
         2763, 2228, 2299, 2290, 3075, 2223, 3051, 2412, 2213]), text=' Sadik , der in ihren Herzen wie in seinem eigenen las , hatte immer seine Freude daran gehabt , dem Wachstum ihrer gegenseitigen Neigung zuzusehen .', dec_out=None, dec_state=None, timestamp=[], alignments=None, frame_confidence=None, token_confidence=None, word_confidence=None, length=0, y=None, lm_state=None, lm_scores=None, ngram_lm_state=None, tokens=None, last_token=None, token_duration=None, last_frame=None),
 Hypothesis(score=0.0, y_sequence=tensor([2284, 2267, 2275, 2222, 2296, 2308, 2291, 2629, 2178, 2232, 2256, 2267,
         2965, 2194, 2262, 2213, 2569,

## Export

In [109]:
from export import exportable_forward, stft
import types


canary.preprocessor.featurizer.forward = types.MethodType(exportable_forward, canary.preprocessor.featurizer)
canary.preprocessor.featurizer.stft = types.MethodType(stft, canary.preprocessor.featurizer)

In [5]:
canary.preprocessor.export(
    "canary/preprocessor.onnx"
)

[NeMo I 2026-01-06 21:49:47 nemo_logging:393] Successfully exported AudioToMelSpectrogramPreprocessor to canary/preprocessor.onnx


(['canary/preprocessor.onnx'],
 ['nemo.collections.asr.modules.audio_preprocessing.AudioToMelSpectrogramPreprocessor exported to ExportFormat.ONNX'])

In [4]:
canary.encoder.export("canary/encoder.onnx")

[NeMo I 2026-01-06 21:49:45 nemo_logging:393] Successfully exported ConformerEncoder to canary/encoder.onnx


(['canary/encoder.onnx'],
 ['nemo.collections.asr.modules.conformer_encoder.ConformerEncoder exported to ExportFormat.ONNX'])

In [26]:
batch_size, seq_len, _ = canary.encoder.input_example()[0].shape

In [82]:
import torch

example_input = torch.randn(batch_size, seq_len, 512)
dynamic_axes = {"encoder_output": {0: "batch_size", 1: "seq_len"}}
torch.onnx.export(
    canary.encoder_decoder_proj,
    (example_input,),
    "canary/encoder_decoder_proj.onnx",
    dynamic_axes=dynamic_axes,
    input_names = ["encoder_output"],
    output_names = ["proj_output"],
    opset_version=17
)

In [56]:
canary.transf_decoder.export("canary/transf_decoder.onnx")

[NeMo I 2026-01-06 23:11:28 nemo_logging:393] Successfully exported TransformerDecoderNM to canary/transf_decoder.onnx


(['canary/transf_decoder.onnx'],
 ['nemo.collections.asr.modules.transformer.transformer.TransformerDecoderNM exported to ExportFormat.ONNX'])

In [74]:
import onnxruntime as ort

In [75]:
encoder_session = ort.InferenceSession("canary/encoder.onnx")
preprocessor_session = ort.InferenceSession("canary/preprocessor.onnx")
proj_session = ort.InferenceSession("canary/encoder_decoder_proj.onnx")
decoder_session = ort.InferenceSession("canary/transf_decoder.onnx")

In [76]:
import librosa
import numpy as np

audio, sr = librosa.load("test_audio.wav", sr=16000)


In [77]:
preprocessed, length = preprocessor_session.run(None, {"input_signal": audio.astype(np.float32).reshape(1, -1), "length": np.array([len(audio)], dtype=np.int64)})

In [78]:
encoded, encoded_length = encoder_session.run(None, {"audio_signal": preprocessed, "length": length})

In [84]:
encoded_mask = np.arange(encoded.shape[1])[None, :] < encoded_length[:, None]

In [79]:
projected, = proj_session.run(None, {"encoder_output": encoded.transpose(0,2,1)})

# Decoder Export

In [109]:
encoded = torch.load("encoded8.pt", weights_only=False)

In [173]:
import types
def _exportable_get_memory_states(self, decoder_states, decoder_mems, i=0):
        past_mem = decoder_mems[i]
        if past_mem.shape[1] > 0:
            memory_states = torch.cat((past_mem, decoder_states), dim=1)
            return memory_states
        else:
            return decoder_states
canary.transf_decoder.decoder._get_memory_states = types.MethodType(_exportable_get_memory_states, canary.transf_decoder.decoder)


In [181]:
from nemo.collections.common.parts.transformer_utils import form_attention_mask

def exportable_forward(
    self,
    decoder_states,
    decoder_mask,
    encoder_states,
    encoder_mask,
    decoder_mems,
):
    # 1. Create masks
    # Ensure form_attention_mask uses torch operations (no numpy/python loops)
    decoder_attn_mask = form_attention_mask(decoder_mask, diagonal=self.diagonal)
    encoder_attn_mask = form_attention_mask(encoder_mask)

    # 2. Process layers
    new_mems = []

    # Layer 0: Embedding memory
    memory_states = self._get_memory_states(decoder_states, decoder_mems, 0)
    new_mems.append(memory_states.unsqueeze(0))

    curr_states = decoder_states
    for i, layer in enumerate(self.layers):
        # print(f"curr_states: {curr_states[:2,:2,:2]}")
        # print(f"decoder_attn_mask: {decoder_attn_mask[:2,:2,:2]}")
        # print(f"memory_states: {memory_states[:2,:2,:2]}")
        # print(memory_states.shape)

        curr_states = layer(curr_states, decoder_attn_mask, memory_states, encoder_states, encoder_attn_mask)
        # print(f"curr_states: {curr_states[:2,:2,:2]}")
        # print(f"curr_states shape: {curr_states.shape}")
        # Memory for layer i+1
        memory_states = self._get_memory_states(curr_states, decoder_mems, i + 1)
        new_mems.append(memory_states.unsqueeze(0))

    if self.final_layer_norm is not None:
        curr_states = self.final_layer_norm(curr_states)
        memory_states = self._get_memory_states(curr_states, decoder_mems, len(self.layers) + 1)
        new_mems.append(memory_states.unsqueeze(0))

    # Concatenate all memories into one tensor for the output
    # Shape: [6, B, L_total, H]
    combined_mems = torch.cat(new_mems,dim=0)
    return combined_mems


canary.transf_decoder.decoder.forward = types.MethodType(exportable_forward, canary.transf_decoder.decoder)

In [182]:
class TransformerDecoderWrapper(torch.nn.Module):
    def __init__(self, canary):
        super().__init__()
        self.decoder = canary.transf_decoder.decoder
        self.embedding = canary.transf_decoder.embedding
        self.num_states = canary.transf_decoder.num_states
        self.token_classifier = canary.decoding.log_softmax_module
    def forward(self,
    input_ids,      # [B, CurrSeqLen]
    decoder_mask,   # [B, TotalSeqLen] (Standard for causal masking)
    encoder_embeddings,
    encoder_mask,
    decoder_mems,   # [num_states, B, MemLen, H]
    start_pos, # scalar tensor
    ):
        # print(f"decoder_input_ids: {input_ids}")
        decoder_embeddings = self.embedding.forward(input_ids=input_ids, start_pos = start_pos.item())
        # print(decoder_embeddings[:2,:2,:2])
        # Decode
        combined_mems = self.decoder(
            decoder_states=decoder_embeddings,
            decoder_mask=decoder_mask,
            encoder_states=encoder_embeddings,
            encoder_mask=encoder_mask,
            decoder_mems=decoder_mems,
        )
        with self.token_classifier.with_log_softmax_enabled(False) as clf:
            logits = clf(hidden_states = combined_mems[-1][:, -1:])
        return logits, combined_mems


In [183]:
wrapper = TransformerDecoderWrapper(canary).eval()

In [184]:
decoder_mask = torch.ones_like(decoder_input_ids)

In [185]:
with torch.inference_mode():
    logits, memories = wrapper(input_ids= decoder_input_ids, decoder_mask=decoder_mask, encoder_embeddings=encoded, encoder_mask = encoded_mask, decoder_mems = torch.zeros((6, len(audio_batch), 0, 1024)), start_pos = torch.tensor(0))

In [186]:
true_logits = torch.load("true_logits.pt", weights_only=False)

In [267]:
decoded_sequence = []
input_ids = decoder_input_ids.clone()
memories = torch.zeros((6, len(audio_batch), 0, 1024))
pad_id = 2
eos_id = 3
pad_profile = torch.zeros(len(audio_batch),1)

with torch.inference_mode():
    for i in range(50):
        print(i)
        decoder_mask = (input_ids != pad_id).float()
        logits, memories = wrapper(input_ids= input_ids, decoder_mask=decoder_mask, encoder_embeddings=encoded, encoder_mask = encoded_mask, decoder_mems = memories, start_pos = torch.tensor(i))
        input_ids = logits[:,-1].argmax(dim=-1, keepdim=True)
        input_ids = pad_id*pad_profile+input_ids*(1 - pad_profile)
        pad_profile = torch.max(pad_profile, (input_ids == eos_id).long())
        decoded_sequence.append(input_ids)

0
1


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [264]:
input_ids

tensor([[2228],
        [   3],
        [   3],
        [   3],
        [3051],
        [   3],
        [   3],
        [   3]])

In [251]:
pad_profile

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [252]:
sequences = torch.hstack(decoded_sequence)

In [253]:
sequences.shape

torch.Size([8, 100])

In [254]:
for sequence in sequences:
    print(canary.tokenizer.ids_to_text(sequence.squeeze().tolist()))

 Sadik , der in ihren Herzen wie in seinem eigenen las , hatte immer seine Freude daran gehabt , dem Wachstum ihrer gegenseitigen Neigung zuzusehen .<|endoftext|><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
 Kein Falsch ist an ihnen und kein Flitter . Ein Wort eine Enziansblüte im Gebirge .<|endoftext|><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
 Vierzig Jahre war Conrad Ferdinand Meyer, als er sein erstes Buch, ein kleines Buch Gedichte, veröffentlichte.<|endoftext|><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [142]:
sequences = [seq.flatten() for seq in decoded_sequence]

In [143]:
canary.tokenizer.ids_to_text(sequences[0])

' S K V die Was Was D Ja'

In [153]:
def export_decoder_to_onnx(model, onnx_path):
    model.eval()

    # Dummy data for the "Prefill" export (this defines the shape structure)
    B = 1
    H = model.hidden_size
    L_prompt = 9
    L_enc = 20
    num_layers = model.num_states # Ensure this matches your layers + norm

    # Initial input: 9 tokens, empty cache (length 0)
    input_ids = torch.randint(0, model.vocab_size, (B, L_prompt))
    dec_mask = torch.ones(B, L_prompt)
    enc_states = torch.randn(B, L_enc, H)
    enc_mask = torch.ones(B, L_enc)
    past_mems = torch.zeros(B, num_layers, 0, H) # Empty cache

    torch.onnx.export(
        model,
        (input_ids, dec_mask, enc_states, enc_mask, past_mems),
        onnx_path,
        export_params=True,
        opset_version=14,
        input_names=['input_ids', 'decoder_mask', 'encoder_states', 'encoder_mask', 'past_mems'],
        output_names=['logits', 'new_mems'],
        dynamic_axes={
            'input_ids': {0: 'batch', 1: 'curr_seq_len'},
            'decoder_mask': {0: 'batch', 1: 'total_seq_len'},
            'encoder_states': {0: 'batch', 1: 'enc_seq_len'},
            'encoder_mask': {0: 'batch', 1: 'enc_seq_len'},
            'past_mems': {0: 'batch', 2: 'past_seq_len'},
            'logits': {0: 'batch', 1: 'curr_seq_len'},
            'new_mems': {0: 'batch', 2: 'total_seq_len'}
        }
    )

In [154]:
export_decoder_to_onnx(canary.transf_decoder, "canary/transf_decoder.onnx")

TypeError: transpose() received an invalid combination of arguments - got (tuple, int, int), but expected one of:
 * (Tensor input, int dim0, int dim1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor)!, int, int)
 * (Tensor input, name dim0, name dim1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor)!, !int!, !int!)
